In [2]:
import nfl_data_py as nfl
import pandas as pd
import os
import urllib.request
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnnotationBbox
from matplotlib.offsetbox import OffsetImage
from PIL import Image
import numpy as np
from io import BytesIO
import requests
# ML Libraries
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
#import hvplot.pandas
import plotly.express as px

# Graphing 
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.pyplot import figure

In [3]:
#Import Data
nfl_data = nfl.import_schedules([2023])
original_df = nfl_data.groupby('week')

In [4]:
# Select relevant columns for team vs. team DataFrame
team_vs_team_df = nfl_data[['week', 'away_team', 'away_score', 'home_team', 'home_score']]


In [5]:
# Get data from games
weekly_nfl = nfl.import_weekly_data([2023, 2022, 2021,2020,2019,2018,2017,2016])
weekly_nfl_two = nfl.import_weekly_data([2024])
weekly_nfl.head()

Downcasting floats.
Downcasting floats.


,player_id,player_name,player_display_name,position,position_group,headshot_url,recent_team,season,week,season_type,...,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr
0,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/private/f_aut...,NYJ,2023,1,REG,...,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,0.0,0.0
1,00-0024243,M.Lewis,Marcedes Lewis,TE,TE,https://static.www.nfl.com/image/private/f_aut...,CHI,2023,4,REG,...,0.0,0.483465,0,0.0,0.031250,-0.012397,0.038197,0.0,0.8,1.8
2,00-0024243,M.Lewis,Marcedes Lewis,TE,TE,https://static.www.nfl.com/image/private/f_aut...,CHI,2023,7,REG,...,1.0,1.437224,0,3.2,0.034483,0.094340,0.117762,0.0,1.6,2.6
3,00-0024243,M.Lewis,Marcedes Lewis,TE,TE,https://static.www.nfl.com/image/private/f_aut...,CHI,2023,11,REG,...,0.0,-0.547367,0,0.0,0.045455,0.036885,0.094001,0.0,0.0,0.0
4,00-0024243,M.Lewis,Marcedes Lewis,TE,TE,https://static.www.nfl.com/image/private/f_aut...,CHI,2023,14,REG,...,0.0,-0.190257,0,0.0,0.033333,-0.012987,0.040909,0.0,0.4,1.4


In [6]:
#clean up data to show data columns for each game
columns = [ 'season','week','recent_team','opponent_team','passing_epa',
           'passing_yards', 'rushing_epa','rushing_yards','receiving_epa'
           ]


In [7]:
#create Df
games_df = weekly_nfl[columns]
games_df.head()

,season,week,recent_team,opponent_team,passing_epa,passing_yards,rushing_epa,rushing_yards,receiving_epa
0,2023,1,NYJ,BUF,-2.03196,0.0,NaN,0.0,NaN
1,2023,4,CHI,DEN,NaN,0.0,NaN,0.0,0.483465
2,2023,7,CHI,LV,NaN,0.0,NaN,0.0,1.437224
3,2023,11,CHI,DET,NaN,0.0,NaN,0.0,-0.547367
4,2023,14,CHI,DET,NaN,0.0,NaN,0.0,-0.190257


In [8]:
# Groupby team and week offense stats
team_by_week = games_df.groupby(['week','season', 'recent_team'])
team_by_week.head()
team_stats_per_game = team_by_week.sum()
team_stats_per_game.index.set_names(['week','season', 'team_offense'], inplace=True)
team_stats_per_game = team_stats_per_game.reset_index()
#team_stats_per_game_off = team_stats_per_game.drop(columns='index')

team_stats_per_game_off = team_stats_per_game.rename(columns={'team_defense': 'team_offense'})
# List of columns to add '_def' to
columns_to_suffix = ['passing_epa',
           'passing_yards', 'rushing_epa','rushing_yards','receiving_epa']

team_stats_per_game_off = team_stats_per_game_off.rename(columns={col: col + '_offense' for col in columns_to_suffix})

team_stats_per_game_off.tail()

,week,season,team_offense,passing_epa_offense,passing_yards_offense,rushing_epa_offense,rushing_yards_offense,receiving_epa_offense
4377,22,2021,LA,4.572897,283.0,-8.289258,43.0,9.025712
4378,22,2022,KC,12.383465,182.0,4.106593,158.0,13.844340
4379,22,2022,PHI,12.892024,304.0,-2.238963,115.0,17.161890
4380,22,2023,KC,6.061147,333.0,-7.526628,130.0,13.647686
4381,22,2023,SF,9.144471,276.0,-9.394331,110.0,12.717166


In [9]:
# Groupby team and week defense stats
team_by_week_def = games_df.groupby(['week','season', 'opponent_team'])
team_by_week_def.head()
team_stats_per_game_def = team_by_week_def.sum()

# List of columns to add '_def' to
columns_to_suffix_def = ['passing_epa',
           'passing_yards', 'rushing_epa','rushing_yards','receiving_epa']

# Add '_def' suffix to the specified columns
team_stats_per_game_def = team_stats_per_game_def.rename(columns={col: col + '_def' for col in columns_to_suffix_def})
team_stats_per_game_def.index.set_names(['week','season', 'team_defense'], inplace=True)
team_stats_per_game_def = team_stats_per_game_def.reset_index()


In [10]:
team_stats_per_game_def.tail()

,week,season,team_defense,passing_epa_def,passing_yards_def,rushing_epa_def,rushing_yards_def,receiving_epa_def
4377,22,2021,LA,-5.851192,269.0,-1.325293,79.0,5.873084
4378,22,2022,KC,12.892024,304.0,-2.238963,115.0,17.161890
4379,22,2022,PHI,12.383465,182.0,4.106593,158.0,13.844340
4380,22,2023,KC,9.144471,276.0,-9.394331,110.0,12.717166
4381,22,2023,SF,6.061147,333.0,-7.526628,130.0,13.647686


In [11]:
# merge both dfs so it shows offense and defense stats per game
team_total_df = team_stats_per_game.merge(team_stats_per_game_def, left_on='team_offense', right_on='team_defense')
# Filter rows where 'team_offense' is equal to 'team_defense'
team_total_df = team_total_df[team_total_df['team_offense'] == team_total_df['team_defense']]
#remove duplicates
team_total_df = team_total_df[(team_total_df['team_offense'] == team_total_df['team_defense']) & (team_total_df['week_x'] == team_total_df['week_y'])]
team_total_df.head()

,week_x,season_x,team_offense,passing_epa,passing_yards,rushing_epa,rushing_yards,receiving_epa,week_y,season_y,team_defense,passing_epa_def,passing_yards_def,rushing_epa_def,rushing_yards_def,receiving_epa_def
0,1,2016,ARI,8.298649,271.0,2.098659,92.0,12.472542,1,2016,ARI,13.317494,264.0,-5.738217,106.0,17.886431
1,1,2016,ARI,8.298649,271.0,2.098659,92.0,12.472542,1,2017,ARI,-0.990408,292.0,-8.114964,82.0,4.320647
2,1,2016,ARI,8.298649,271.0,2.098659,92.0,12.472542,1,2018,ARI,13.331767,255.0,2.684754,182.0,14.428679
3,1,2016,ARI,8.298649,271.0,2.098659,92.0,12.472542,1,2019,ARI,11.415977,385.0,-8.551727,116.0,19.748468
4,1,2016,ARI,8.298649,271.0,2.098659,92.0,12.472542,1,2020,ARI,0.376996,259.0,-3.209652,123.0,5.359482


In [12]:
# create a copy of team_total_df for home team and away team

# Make the first copy of the original DataFrame
home_team_df = team_total_df.copy()

# Make a second copy from the first copy
away_team_df = home_team_df.copy()

In [13]:
#make a home_team df
# Add '_home' to each column name
for column in home_team_df.columns:
    new_column_name_home = column + '_home'
    home_team_df.rename(columns={column: new_column_name_home}, inplace=True)
    
home_team_df.head()


,week_x_home,season_x_home,team_offense_home,passing_epa_home,passing_yards_home,rushing_epa_home,rushing_yards_home,receiving_epa_home,week_y_home,season_y_home,team_defense_home,passing_epa_def_home,passing_yards_def_home,rushing_epa_def_home,rushing_yards_def_home,receiving_epa_def_home
0,1,2016,ARI,8.298649,271.0,2.098659,92.0,12.472542,1,2016,ARI,13.317494,264.0,-5.738217,106.0,17.886431
1,1,2016,ARI,8.298649,271.0,2.098659,92.0,12.472542,1,2017,ARI,-0.990408,292.0,-8.114964,82.0,4.320647
2,1,2016,ARI,8.298649,271.0,2.098659,92.0,12.472542,1,2018,ARI,13.331767,255.0,2.684754,182.0,14.428679
3,1,2016,ARI,8.298649,271.0,2.098659,92.0,12.472542,1,2019,ARI,11.415977,385.0,-8.551727,116.0,19.748468
4,1,2016,ARI,8.298649,271.0,2.098659,92.0,12.472542,1,2020,ARI,0.376996,259.0,-3.209652,123.0,5.359482


In [14]:
#make a away_team df
# Add '_away' to each column name
for column in away_team_df.columns:
    new_column_name_away = column + '_away'
    away_team_df.rename(columns={column: new_column_name_away}, inplace=True)
away_team_df.tail()
#away_team.to_excel('ex.xlsx', index=False)


,week_x_away,season_x_away,team_offense_away,passing_epa_away,passing_yards_away,rushing_epa_away,rushing_yards_away,receiving_epa_away,week_y_away,season_y_away,team_defense_away,passing_epa_def_away,passing_yards_def_away,rushing_epa_def_away,rushing_yards_def_away,receiving_epa_def_away
600513,18,2022,WAS,-7.461784,169.0,-4.547848,151.0,-3.066667,18,2023,WAS,23.005646,309.0,2.093270,131.0,23.005646
600642,18,2023,WAS,-7.128528,157.0,-7.246222,50.0,-0.754869,18,2020,WAS,19.551708,381.0,-3.118225,142.0,25.708466
600643,18,2023,WAS,-7.128528,157.0,-7.246222,50.0,-0.754869,18,2021,WAS,-19.281744,103.0,-7.471216,94.0,-9.882633
600644,18,2023,WAS,-7.128528,157.0,-7.246222,50.0,-0.754869,18,2022,WAS,-14.503402,128.0,-17.298262,64.0,-11.103701
600645,18,2023,WAS,-7.128528,157.0,-7.246222,50.0,-0.754869,18,2023,WAS,23.005646,309.0,2.093270,131.0,23.005646


In [15]:
# Make df with points and stats

# merge home team with home stats
home_team = team_vs_team_df.merge(home_team_df, left_on='home_team', right_on='team_offense_home')
#remove duplicates
home_team = home_team[(home_team['home_team'] ==  home_team['team_offense_home']) & (home_team['week_x_home'] == home_team['week'])]
home_team.tail(10)

,week,away_team,away_score,home_team,home_score,week_x_home,season_x_home,team_offense_home,passing_epa_home,passing_yards_home,...,rushing_yards_home,receiving_epa_home,week_y_home,season_y_home,team_defense_home,passing_epa_def_home,passing_yards_def_home,rushing_epa_def_home,rushing_yards_def_home,receiving_epa_def_home
287631,18,DEN,14.0,LV,27.0,18,2021,LV,-4.420844,190.0,...,174.0,3.122958,18,2022,LV,1.010589,202.0,7.964468,168.0,7.345337
287632,18,DEN,14.0,LV,27.0,18,2021,LV,-4.420844,190.0,...,174.0,3.122958,18,2023,LV,0.353189,272.0,-2.951732,48.0,9.411932
287633,18,DEN,14.0,LV,27.0,18,2022,LV,-9.471091,219.0,...,99.0,3.817271,18,2016,LV,0.376396,168.0,-10.807101,123.0,0.376396
287634,18,DEN,14.0,LV,27.0,18,2022,LV,-9.471091,219.0,...,99.0,3.817271,18,2021,LV,14.838099,383.0,-5.043431,85.0,20.664579
287635,18,DEN,14.0,LV,27.0,18,2022,LV,-9.471091,219.0,...,99.0,3.817271,18,2022,LV,1.010589,202.0,7.964468,168.0,7.345337
287636,18,DEN,14.0,LV,27.0,18,2022,LV,-9.471091,219.0,...,99.0,3.817271,18,2023,LV,0.353189,272.0,-2.951732,48.0,9.411932
287637,18,DEN,14.0,LV,27.0,18,2023,LV,9.306783,244.0,...,129.0,17.544756,18,2016,LV,0.376396,168.0,-10.807101,123.0,0.376396
287638,18,DEN,14.0,LV,27.0,18,2023,LV,9.306783,244.0,...,129.0,17.544756,18,2021,LV,14.838099,383.0,-5.043431,85.0,20.664579
287639,18,DEN,14.0,LV,27.0,18,2023,LV,9.306783,244.0,...,129.0,17.544756,18,2022,LV,1.010589,202.0,7.964468,168.0,7.345337
287640,18,DEN,14.0,LV,27.0,18,2023,LV,9.306783,244.0,...,129.0,17.544756,18,2023,LV,0.353189,272.0,-2.951732,48.0,9.411932


In [16]:
# merge away team with home stats
away_team = home_team.merge(away_team_df, left_on='away_team', right_on='team_offense_away')
#remove duplicates
away_team = away_team[(away_team['away_team'] ==  away_team['team_offense_away']) & (away_team['week_x_away'] == away_team['week'])]
away_team = away_team.drop_duplicates(subset=['week_x_home','away_team', 'home_team','away_score','home_score'],keep='first')

away_team.to_excel('ex.xlsx', index=False)


In [17]:
# rename df
full_df = away_team.copy()

In [18]:
#full_df.to_excel('training_nfl.xlsx', index=False)


In [19]:
# Create Columns for training
column_train_x = [
   'passing_epa_home','passing_yards_home','rushing_epa_home','rushing_yards_home','receiving_epa_home',
    'passing_epa_def_home','passing_yards_def_home','rushing_epa_def_home','rushing_yards_def_home','receiving_epa_def_home',
       'passing_epa_away','passing_yards_away','rushing_epa_away','rushing_yards_away','receiving_epa_away',
           'passing_epa_def_away','passing_yards_def_away','rushing_epa_def_away','rushing_yards_def_away','receiving_epa_def_away',


]




column_train_y = ['home_score','away_score']

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# model
# Split data into features (X) and target variable (y)
X = full_df[column_train_x]
y = full_df[column_train_y]

# Encode categorical variables using one-hot encoding if needed

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Create a Linear Regression model
model = LinearRegression()

# Train the model
model.fit(X_train, y_train)

# Make predictions
predictions = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = mean_squared_error(y_test, predictions, squared=False)
r2 = r2_score(y_test, predictions)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R2): {r2}")


Mean Absolute Error (MAE): 8.870206623746638
Mean Squared Error (MSE): 125.96731816390735
Root Mean Squared Error (RMSE): 11.21554664559585
R-squared (R2): -0.12256599794295209


In [21]:
#get 2024 averages
#create Df
games_df_two = weekly_nfl_two[columns]
games_df_two.head()

# Groupby team and week offense stats
team_by_week_two = games_df_two.groupby(['week', 'recent_team'])
team_by_week_two.head()
team_stats_per_game_two = team_by_week_two.sum()
team_stats_per_game_two.index.set_names(['week','team_offense'], inplace=True)
team_stats_per_game_two = team_stats_per_game_two.reset_index()
#team_stats_per_game_off = team_stats_per_game.drop(columns='index')

team_stats_per_game_off_two = team_stats_per_game_two.rename(columns={'team_defense': 'team_offense'})
# List of columns to add '_def' to
columns_to_suffix = ['passing_epa',
           'passing_yards', 'rushing_epa','rushing_yards','receiving_epa']

team_stats_per_game_off_two = team_stats_per_game_off_two.rename(columns={col: col + '_offense' for col in columns_to_suffix})
team_stats_per_game_off_two = team_stats_per_game_off_two.drop('season', axis=1)

#team_stats_per_game_off_two.head()




In [22]:
team_stats_per_game_off.head()

,week,season,team_offense,passing_epa_offense,passing_yards_offense,rushing_epa_offense,rushing_yards_offense,receiving_epa_offense
0,1,2016,ARI,8.298649,271.0,2.098659,92.0,12.472542
1,1,2016,ATL,7.936941,334.0,-7.653444,52.0,11.217734
2,1,2016,BAL,3.035203,258.0,-11.255319,83.0,10.121665
3,1,2016,BUF,-1.960813,111.0,-4.708451,65.0,1.535697
4,1,2016,CAR,1.192924,194.0,1.428706,157.0,4.863992


In [23]:
### Create mean of these stats to create test data
#home team
mean_offense = team_stats_per_game_off_two.groupby('team_offense').mean()
mean_offense = mean_offense.drop('week', axis=1)
mean_offense = mean_offense.reset_index()

mean_offense.head()

,team_offense,passing_epa_offense,passing_yards_offense,rushing_epa_offense,rushing_yards_offense,receiving_epa_offense
0,ARI,-1.184850,162.0,3.931950,124.0,8.129718
1,ATL,-11.084852,155.0,-6.224773,89.0,-7.576805
2,BAL,-0.768779,260.0,3.411777,168.0,3.730892
3,BUF,10.156256,185.5,1.018189,119.0,13.719385
4,CAR,-20.260334,122.5,-4.372826,74.0,-14.021781


In [24]:
# Groupby team and week defense stats
team_by_week_def_two = games_df_two.groupby(['week', 'opponent_team'])
team_stats_per_game_def_two = team_by_week_def_two.sum()
team_stats_per_game_def_two.index.set_names(['week','team_offense'], inplace=True)
team_stats_per_game_def_two = team_stats_per_game_def_two.reset_index()
team_stats_per_game_def_two = team_stats_per_game_def_two.rename(columns={'team_offense': 'team_defense'})

# List of columns to add '_def' to
columns_to_suffix_def = ['passing_epa',
           'passing_yards', 'rushing_epa','rushing_yards','receiving_epa']

# Add '_def' suffix to the specified columns
team_stats_per_game_def_two = team_stats_per_game_def_two.rename(columns={col: col + '_def' for col in columns_to_suffix_def})
team_stats_per_game_def_two = team_stats_per_game_def_two.drop('season', axis=1)

team_stats_per_game_def_two.head()

,week,team_defense,passing_epa_def,passing_yards_def,rushing_epa_def,rushing_yards_def,receiving_epa_def
0,1,ARI,9.161098,232.0,3.497710,130.0,15.683219
1,1,ATL,-1.352277,156.0,-5.234161,137.0,3.547840
2,1,BAL,11.084406,291.0,-0.872501,72.0,14.509170
3,1,BUF,-1.184850,162.0,3.931950,124.0,8.129718
4,1,CAR,7.993491,200.0,4.767348,180.0,8.904821


In [25]:
#defense team
# Split the DataFrame into non-numerical and numerical columns
mean_defense = team_stats_per_game_def_two.groupby('team_defense').mean()
# Remove the week column
mean_defense = mean_defense.drop('week', axis=1)
mean_defense = mean_defense.reset_index()

mean_defense.head()


,team_defense,passing_epa_def,passing_yards_def,rushing_epa_def,rushing_yards_def,receiving_epa_def
0,ARI,9.161098,232.0,3.497710,130.0,15.683219
1,ATL,-1.352277,156.0,-5.234161,137.0,3.547840
2,BAL,4.321736,283.5,-4.102555,49.5,11.886948
3,BUF,-8.794344,193.5,0.165535,131.5,2.561988
4,CAR,4.713571,165.0,3.903278,199.5,7.942597


In [26]:
# Merge on defense vs offense per team
test_data_df = pd.merge(mean_defense, mean_offense, left_on='team_defense',right_on='team_offense', how='left')
test_data_df.head()

,team_defense,passing_epa_def,passing_yards_def,rushing_epa_def,rushing_yards_def,receiving_epa_def,team_offense,passing_epa_offense,passing_yards_offense,rushing_epa_offense,rushing_yards_offense,receiving_epa_offense
0,ARI,9.161098,232.0,3.497710,130.0,15.683219,ARI,-1.184850,162.0,3.931950,124.0,8.129718
1,ATL,-1.352277,156.0,-5.234161,137.0,3.547840,ATL,-11.084852,155.0,-6.224773,89.0,-7.576805
2,BAL,4.321736,283.5,-4.102555,49.5,11.886948,BAL,-0.768779,260.0,3.411777,168.0,3.730892
3,BUF,-8.794344,193.5,0.165535,131.5,2.561988,BUF,10.156256,185.5,1.018189,119.0,13.719385
4,CAR,4.713571,165.0,3.903278,199.5,7.942597,CAR,-20.260334,122.5,-4.372826,74.0,-14.021781


In [27]:
# create DF for upcoming week
# create stats to show upcpoming games

#2024 update to get 2024 schedule
nfl_data_two = nfl.import_schedules([2024])

grouped = nfl_data_two.groupby('week')
# Select the group corresponding to week 6
week_data = grouped.get_group(2)
prior_week = grouped.get_group(1)
get_teams =['home_team','away_team','home_score','away_score']
schedule_week_df = week_data[get_teams]
schedule_week_df.reset_index(drop=True, inplace=True)
schedule_week_df.head()

,home_team,away_team,home_score,away_score
0,MIA,BUF,10.0,31.0
1,BAL,LV,23.0,26.0
2,CAR,LAC,3.0,26.0
3,DAL,NO,19.0,44.0
4,DET,TB,16.0,20.0


In [28]:
# create a copy of team_total_df for home team and away team for testing

# Make the first copy of the original DataFrame
home_team_test = test_data_df.copy()

# Make a second copy from the first copy
away_team_test = home_team_test.copy()

In [29]:
# Add '_home' to each column name
for column in home_team_test.columns:
    new_column_name_test = column + '_home'
    home_team_test.rename(columns={column: new_column_name_test}, inplace=True)
    
home_team_test.head()

,team_defense_home,passing_epa_def_home,passing_yards_def_home,rushing_epa_def_home,rushing_yards_def_home,receiving_epa_def_home,team_offense_home,passing_epa_offense_home,passing_yards_offense_home,rushing_epa_offense_home,rushing_yards_offense_home,receiving_epa_offense_home
0,ARI,9.161098,232.0,3.497710,130.0,15.683219,ARI,-1.184850,162.0,3.931950,124.0,8.129718
1,ATL,-1.352277,156.0,-5.234161,137.0,3.547840,ATL,-11.084852,155.0,-6.224773,89.0,-7.576805
2,BAL,4.321736,283.5,-4.102555,49.5,11.886948,BAL,-0.768779,260.0,3.411777,168.0,3.730892
3,BUF,-8.794344,193.5,0.165535,131.5,2.561988,BUF,10.156256,185.5,1.018189,119.0,13.719385
4,CAR,4.713571,165.0,3.903278,199.5,7.942597,CAR,-20.260334,122.5,-4.372826,74.0,-14.021781


In [30]:
# Add '_away' to each column name
for column in away_team_test.columns:
    new_column_name_test_away = column + '_away'
    away_team_test.rename(columns={column: new_column_name_test_away}, inplace=True)
    
away_team_test.head()

,team_defense_away,passing_epa_def_away,passing_yards_def_away,rushing_epa_def_away,rushing_yards_def_away,receiving_epa_def_away,team_offense_away,passing_epa_offense_away,passing_yards_offense_away,rushing_epa_offense_away,rushing_yards_offense_away,receiving_epa_offense_away
0,ARI,9.161098,232.0,3.497710,130.0,15.683219,ARI,-1.184850,162.0,3.931950,124.0,8.129718
1,ATL,-1.352277,156.0,-5.234161,137.0,3.547840,ATL,-11.084852,155.0,-6.224773,89.0,-7.576805
2,BAL,4.321736,283.5,-4.102555,49.5,11.886948,BAL,-0.768779,260.0,3.411777,168.0,3.730892
3,BUF,-8.794344,193.5,0.165535,131.5,2.561988,BUF,10.156256,185.5,1.018189,119.0,13.719385
4,CAR,4.713571,165.0,3.903278,199.5,7.942597,CAR,-20.260334,122.5,-4.372826,74.0,-14.021781


In [31]:
# Make df with points and stats for testing upcoming week

# merge home team with home stats
testing_home = schedule_week_df.merge(home_team_test, left_on='home_team', right_on='team_offense_home')
#remove duplicates
#testing_home = home_team[(home_team['home_team'] ==  home_team['team_offense_home']) & (home_team['week_x_home'] == home_team['week'])]
testing_home.head(1)

,home_team,away_team,home_score,away_score,team_defense_home,passing_epa_def_home,passing_yards_def_home,rushing_epa_def_home,rushing_yards_def_home,receiving_epa_def_home,team_offense_home,passing_epa_offense_home,passing_yards_offense_home,rushing_epa_offense_home,rushing_yards_offense_home,receiving_epa_offense_home
0,MIA,BUF,10.0,31.0,MIA,6.428982,150.5,-5.283032,118.0,8.851244,MIA,-6.040143,281.5,-3.179832,110.0,2.743685


In [32]:
# merge away team with home stats
testing_away = testing_home.merge(away_team_test, left_on='away_team', right_on='team_offense_away')
#remove duplicates
#away_team = away_team[(away_team['away_team'] ==  away_team['team_offense_away']) & (away_team['week_x_away'] == away_team['week'])]
testing_away.head(10)

,home_team,away_team,home_score,away_score,team_defense_home,passing_epa_def_home,passing_yards_def_home,rushing_epa_def_home,rushing_yards_def_home,receiving_epa_def_home,...,passing_yards_def_away,rushing_epa_def_away,rushing_yards_def_away,receiving_epa_def_away,team_offense_away,passing_epa_offense_away,passing_yards_offense_away,rushing_epa_offense_away,rushing_yards_offense_away,receiving_epa_offense_away
0,MIA,BUF,10.0,31.0,MIA,6.428982,150.5,-5.283032,118.0,8.851244,...,193.5,0.165535,131.5,2.561988,BUF,10.156256,185.5,1.018189,119.0,13.719385
1,BAL,LV,23.0,26.0,BAL,4.321736,283.5,-4.102555,49.5,11.886948,...,195.5,-0.208596,163.5,0.914492,LV,-1.841819,266.5,-12.922684,49.0,10.276772
2,CAR,LAC,3.0,26.0,CAR,4.713571,165.0,3.903278,199.5,7.942597,...,170.5,-9.513897,80.5,-2.918723,LAC,-1.721114,137.0,2.128835,197.5,2.535315
3,DAL,NO,19.0,44.0,DAL,-4.544913,206.0,7.252850,141.5,1.806583,...,230.0,-4.390527,63.0,-8.334380,NO,11.009660,221.5,7.593949,185.0,12.264771
4,DET,TB,16.0,20.0,DET,0.550120,251.0,-1.225923,76.5,6.634991,...,254.0,6.405468,138.5,-2.845899,TB,8.577792,237.0,-3.994170,91.0,14.792474
5,GB,IND,16.0,10.0,GB,1.568339,241.0,-1.763054,142.0,5.843225,...,178.0,-3.492298,237.0,12.599012,IND,-5.536810,208.0,4.661525,122.0,-2.028711
6,JAX,CLE,13.0,18.0,JAX,2.783057,262.0,-1.274662,103.0,7.150994,...,199.5,2.152085,114.5,0.111014,CLE,-10.936546,177.5,2.147306,109.0,-3.101339
7,MIN,SF,23.0,17.0,MIN,-11.014616,252.5,-3.626834,88.0,-1.600170,...,235.5,0.536525,107.0,1.725724,SF,2.488087,275.0,-1.512631,141.0,10.357626
8,NE,SEA,20.0,23.0,NE,2.561271,245.5,-2.687689,58.0,3.283985,...,143.5,1.644368,142.0,-5.906603,SEA,2.250231,249.0,-3.223367,96.0,4.989396
9,TEN,NYJ,17.0,24.0,TEN,-5.772955,134.5,-0.244514,92.5,-1.428914,...,211.5,1.801323,155.0,5.552352,NYJ,4.061307,189.5,-0.891510,84.5,4.705263


In [33]:
# rename df
full_test = testing_away.copy()
#full_test = full_test.reset_index()

full_test = full_test.rename(columns={
    'passing_epa_offense_away': 'passing_epa_away',
    'passing_yards_offense_away': 'passing_yards_away',
    'receiving_epa_offense_away': 'receiving_epa_away',
    'rushing_epa_offense_away': 'rushing_epa_away',
    'rushing_yards_offense_away': 'rushing_yards_away',
     'passing_epa_offense_home': 'passing_epa_home',
    'passing_yards_offense_home': 'passing_yards_home',
    'receiving_epa_offense_home': 'receiving_epa_home',
    'rushing_epa_offense_home': 'rushing_epa_home',
    'rushing_yards_offense_home': 'rushing_yards_home',
})
full_test.dtypes

home_team                  object
away_team                  object
home_score                float64
away_score                float64
team_defense_home          object
passing_epa_def_home      float32
passing_yards_def_home    float32
rushing_epa_def_home      float32
rushing_yards_def_home    float32
receiving_epa_def_home    float32
team_offense_home          object
passing_epa_home          float32
passing_yards_home        float32
rushing_epa_home          float32
rushing_yards_home        float32
receiving_epa_home        float32
team_defense_away          object
passing_epa_def_away      float32
passing_yards_def_away    float32
rushing_epa_def_away      float32
rushing_yards_def_away    float32
receiving_epa_def_away    float32
team_offense_away          object
passing_epa_away          float32
passing_yards_away        float32
rushing_epa_away          float32
rushing_yards_away        float32
receiving_epa_away        float32
dtype: object

In [34]:
column_test_x = ['passing_epa_def_away', 'passing_yards_def_away',
       'rushing_epa_def_away', 'rushing_yards_def_away',
       'receiving_epa_def_away','passing_epa_away', 'passing_yards_away',
    'rushing_epa_away', 'rushing_yards_away','receiving_epa_away',
     'passing_epa_def_home', 'passing_yards_def_home',
       'rushing_epa_def_home', 'rushing_yards_def_home',
       'receiving_epa_def_home','passing_epa_home', 'passing_yards_home',
    'rushing_epa_home', 'rushing_yards_home',
       'receiving_epa_home',            
                ]

In [98]:
# Now, use the trained model to predict score for df_test
X_test = full_test[column_test_x]
y_pred = model.predict(X_test)

# The predictions are stored in y_pred
print("Predicted Scores:")
print(y_pred)

Predicted Scores:
[[24.636597 22.929565]
 [23.824986 17.302462]
 [24.736423 18.36787 ]
 [28.785284 18.061321]
 [18.519379 22.298866]
 [26.954716 23.033728]
 [26.54161  22.873556]
 [23.523125 16.79369 ]
 [24.445927 22.27325 ]
 [24.831602 20.05584 ]
 [19.79326  20.154663]
 [26.196234 21.242805]
 [22.756317 20.012497]
 [25.933979 22.744621]
 [24.270618 23.255402]
 [24.860762 20.436237]]


/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


In [43]:
# add predicted scores into df: 
predict_cols = ['home_team','away_team']
df_predictions = full_test[predict_cols]
scores_df = pd.DataFrame(y_pred, columns=['home_score', 'away_score'])
df_predictions = pd.concat([df_predictions, scores_df], axis=1)
df_predictions['Over/Under'] = df_predictions['home_score'] + df_predictions['away_score']
# Assuming 'home_score', 'away_score', 'home_team', and 'away_team' are columns in your DataFrame
df_predictions['Favorite'] = np.where(df_predictions['home_score'] > df_predictions['away_score'], df_predictions['home_team'], df_predictions['away_team'])
df_predictions['Spread'] = df_predictions['away_score'] - df_predictions['home_score']
df_predictions['spread'] = df_predictions['Spread'].apply(lambda x: '-' + str(x) if x >= 0 else str(x))
df_predictions = df_predictions.drop('Spread', axis=1)

df_predictions.head(17)

,home_team,away_team,home_score,away_score,Over/Under,Favorite,spread
0,MIA,BUF,24.555304,20.388472,44.943775,MIA,-4.166831970214844
1,BAL,LV,23.916855,20.488647,44.405502,BAL,-3.4282073974609375
2,CAR,LAC,25.161737,20.178820,45.340557,CAR,-4.982917785644531
3,DAL,NO,24.415163,19.673813,44.088974,DAL,-4.741350173950195
4,DET,TB,24.126490,20.198811,44.325302,DET,-3.9276790618896484
5,GB,IND,23.636423,19.768656,43.405079,GB,-3.867767333984375
6,JAX,CLE,24.051773,19.827868,43.879639,JAX,-4.223905563354492
7,MIN,SF,24.565823,19.318516,43.884338,MIN,-5.247306823730469
8,NE,SEA,24.094368,20.294418,44.388786,NE,-3.7999496459960938
9,TEN,NYJ,23.753361,20.328094,44.081455,TEN,-3.4252662658691406


In [304]:
# Convert the 'spread' column to float
df_predictions['spread'] = pd.to_numeric(df_predictions['spread'], errors='coerce')
# Round the numeric columns to 1 decimal place
df_predictions['home_score'] = df_predictions['home_score'].round(2)
df_predictions['away_score'] = df_predictions['away_score'].round(2)
df_predictions['Over/Under'] = df_predictions['Over/Under'].round(2)
df_predictions['spread'] = df_predictions['spread'].round(2)
# Set display precision
pd.set_option('display.float_format', '{:.1f}'.format)



df_predictions.head(16)

,home_team,away_team,home_score,away_score,Over/Under,Favorite,spread
0,MIA,BUF,23.6,22.7,46.3,MIA,-0.9
1,BAL,LV,25.4,16.9,42.3,BAL,-8.5
2,CAR,LAC,20.5,20.4,41.0,CAR,-0.1
3,DAL,NO,24.2,19.4,43.5,DAL,-4.8
4,DET,TB,25.3,21.3,46.6,DET,-4.0
5,GB,IND,24.6,20.7,45.3,GB,-3.9
6,JAX,CLE,21.4,19.0,40.4,JAX,-2.5
7,MIN,SF,21.2,24.3,45.5,SF,-3.1
8,NE,SEA,20.6,20.0,40.7,NE,-0.6
9,TEN,NYJ,20.7,17.4,38.0,TEN,-3.3


In [305]:
#df_predictions.to_excel('week_1_pred.xlsx', index=False)


In [306]:
# Add in actual odds from these games
vegas_df = pd.read_excel('vegas_odds/week_one_vegas_odds.xlsx')
vegas_df.head(16)


,home_team,away_team,Over/Under_Vegas,Favorite_Vegas,Spread_Vegas
0,KC,BAL,46.0,KC,-3.0
1,PHI,GB,48.5,PHI,-2.5
2,CIN,NE,40.5,CIN,-8.5
3,CHI,TEN,45.0,CHI,-3.5
4,MIA,JAX,49.5,MIA,-3.0
5,IND,HOU,49.0,HOU,-2.5
6,BUF,ARI,48.0,BUF,-6.0
7,ATL,PIT,42.0,ATL,-3.0
8,NYG,MIN,41.5,MIN,-1.5
9,NO,CAR,41.5,NO,-4.0


In [307]:
#merge so model and vegas odds are on each game
# Merge DataFrames
model_vegas_df = pd.merge(df_predictions, vegas_df, on=['home_team', 'away_team'])
model_vegas_df['Over/Under_Difference'] = model_vegas_df['Over/Under'] - model_vegas_df['Over/Under_Vegas']


model_vegas_df.head(16)


,home_team,away_team,home_score,away_score,Over/Under,Favorite,spread,Over/Under_Vegas,Favorite_Vegas,Spread_Vegas,Over/Under_Difference


In [147]:
from flask import Flask, render_template
app = Flask(__name__)

@app.route('/')
def display_dataframe():
    return df_predictions.to_html()

if __name__ == '__main__':
    app.run()










 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [06/Sep/2024 15:44:53] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Sep/2024 15:44:54] "GET /favicon.ico HTTP/1.1" 404 -
